In [1]:
# GET RSSI DATA

import serial
import time

In [2]:
# autoconnect
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)
ut_port = p[0] #working perfectly on my Huawei Modem

/dev/ttyUSB2 - HUAWEI Mobile
/dev/ttyUSB0 - HUAWEI Mobile


In [3]:
#ut_port='/dev/ttyUSB0' #by default, in first after boot will be USB0
ut=serial.Serial(ut_port, 115200, timeout=0)
print(ut.name)

/dev/ttyUSB0


In [4]:
#check OK
cmd_ok='AT\r'
ut.write(cmd_ok.encode())
print('Getting device ready ... ')

Getting device ready ... 


In [5]:
ut_ok=ut.readline() #blank
ut_ok=ut.readline() #OK
ut_ok=ut_ok.decode('utf-8')
oke=ut_ok[0:2]
if oke=='OK':
    print(oke+': Your device is ready')
else:
    print(oke+': Your device is not ready. Please try again')

OK: Your device is ready


In [6]:
#checking device IMEI
ut.reset_output_buffer()
cmd_imei='AT+CGSN\r'
ut.write(cmd_imei.encode())
print('Getting device information ... ')

Getting device information ... 


In [7]:
ut_cgsn=ut.readline() #blank
ut_cgsn=ut.readline() #IMEI
ut_cgsn=ut_cgsn.decode('utf-8')

In [8]:
imei=ut_cgsn[0:15] #unique id upload to server
imei=int(imei) #convert to integer as spec
print("IMEI : " + str(imei))

IMEI : 867497013638036


In [9]:
#getting RSSI data
ut.reset_output_buffer()
cmd_csq='AT+CSQ\r'
ut.write(cmd_csq.encode())
print('Getting RSSI ... ')

Getting RSSI ... 


In [10]:
epoch=int(time.time())

In [11]:
ut_csq=ut.readline() #blank
ut_csq=ut.readline() #OK
ut_csq=ut.readline() #blank
ut_csq=ut.readline() #CSQ
ut_csq=ut_csq.decode('utf-8')

In [12]:
colon=ut_csq.find(',')
rssi=ut_csq[colon-2:colon]
ber=ut_csq[colon+1:colon+3]
rssi=int(rssi)
ber=int(ber)
csq=float(rssi-95) #meraki formula dbm https://documentation.meraki.com/MR/Monitoring_and_Reporting/Location_Analytics
print("Timestamp: " + str(epoch))
print("Signal strength: " + str(csq) + " dBm")
print("RSSI: " + str(rssi))
print("BER: " + str(ber))

Timestamp: 1592453407
Signal strength: -95.0 dBm
RSSI: 0
BER: 99


In [13]:
# importing the requests library 
import requests 
  
# defining the api-endpoint  
API_ENDPOINT = "http://pastebin.com/api/api_post.php"
  
# your API key here 
API_KEY = "8e2c6e885cc7441ca3bf3885063c97a5"
  
# your source code here 
json_body = {'csq':csq, 'id':imei, 'ts':epoch}
pasted = str(json_body)

# data to be sent to api 
data = {'api_dev_key':API_KEY, 
        'api_option':'paste', 
        'api_paste_code':pasted, 
        'api_paste_format':'python'} 
  
# sending post request and saving response as response object 
r = requests.post(url = API_ENDPOINT, data = data) 
  
# extracting response text  
pastebin_url = r.text 
print("The pastebin URL is:%s"%pastebin_url) 

The pastebin URL is:https://pastebin.com/h4FRAKMz


In [14]:
print(pasted)

{'csq': -95.0, 'id': 867497013638036, 'ts': 1592453407}
